In [618]:
import pandas as pd 
import os
import numpy as np

os.chdir(r'C:\Users\nicho\OneDrive\Desktop\Bitcoin Regresssion Project')
BTC = pd.read_csv(r'BTC-USD.csv' , parse_dates = ['Date'], index_col=['Date'])
SNP = pd.read_csv(r'Standard&PoorsHistorical.csv', parse_dates = ['Date'], index_col=['Date'])
Margin = pd.read_csv(r'margin-statistics.csv')
CPI = pd.read_csv(r'CPIHistoricalData.csv', parse_dates = ['Date'], index_col= ['Date'])
BTC.head(), Margin.head(), SNP.head(), CPI.head()

(            Symbol     Open     High      Low    Close  Volume BTC  \
 Date                                                                 
 2020-04-10  BTCUSD  7315.25  7315.25  7315.25  7315.25        0.00   
 2020-04-09  BTCUSD  7369.60  7378.85  7115.04  7315.25     2237.13   
 2020-04-08  BTCUSD  7201.81  7432.23  7152.80  7369.60     2483.60   
 2020-04-07  BTCUSD  7336.96  7468.42  7078.00  7201.81     2333.34   
 2020-04-06  BTCUSD  6775.21  7369.76  6771.01  7336.96     3727.47   
 
              Volume USD  
 Date                     
 2020-04-10         0.00  
 2020-04-09  16310014.53  
 2020-04-08  18138080.27  
 2020-04-07  17047120.32  
 2020-04-06  26533750.17  ,
   Year-Month Debit Balances in Customers' Securities Margin Accounts  \
 0    2021-11                                            918,598        
 1    2021-10                                            935,862        
 2    2021-09                                            903,117        
 3    2021-08      

In [619]:
#Dropping irrelevant features

Drop = ['Symbol', 'Open', 'High', 'Low', 'Volume BTC']
BTCDrop = BTC.drop(Drop, axis = 1)

Drop1 = ['Volume', 'Open', 'High', 'Low']
SNPDrop = SNP.drop(Drop1, axis = 1)


Drop3 = ['Open', 'High', 'Low', 'Volume']
CPIDrop = CPI.drop(Drop3, axis = 1)

In [620]:
#Renaming Columns
BTCDrop.rename(columns = {'Close': 'BTC', 
                          'Volume USD': 'Liquidity'}, inplace = True)

SNPDrop.rename(columns = {'Close/Last': 'SNP close'}, inplace = True)

CPIDrop.rename(columns = {'Close/Last': 'CPI close'}, inplace = True)

In [621]:
BTCMonthly = BTCDrop.resample('M').median()
CPIMonthly = CPIDrop.resample('M').median()
SNPMonthly = SNPDrop.resample('M').median()

BTCMonthly.reset_index(inplace = True)
SNPMonthly.reset_index(inplace = True)
CPIMonthly.reset_index(inplace = True)


BTCMonthly['Date'] = BTCMonthly['Date'].dt.strftime('%Y-%m')
SNPMonthly['Date'] = SNPMonthly['Date'].dt.strftime('%Y-%m')
CPIMonthly['Date'] = CPIMonthly['Date'].dt.strftime('%Y-%m')



In [622]:
#Dataframe with Margin feature
MarginMerged = pd.concat([Margin, BTCMonthly, SNPMonthly, CPIMonthly], axis=1, join='inner')

MarginMerged.drop(['Free Credit Balances in Customers\' Cash Accounts', 'Date', 
             'Free Credit Balances in Customers\' Securities Margin Accounts'], axis = 1)

,Year-Month,Debit Balances in Customers' Securities Margin Accounts,BTC,Liquidity,SNP close,CPI close
0,2021-11,"918,598",272.285,5.274223e+04,1294.585,-0.10
1,2021-10,"935,862",337.720,2.913455e+05,1351.950,-0.10
2,2021-09,"903,117",432.500,1.813676e+05,1396.530,0.30
3,2021-08,"911,545",412.140,2.164131e+05,1387.570,0.30
4,2021-07,"844,324",406.180,3.509556e+05,1330.660,0.30
5,2021-06,"882,103",414.680,4.153307e+05,1325.510,-0.30
6,2021-05,"861,626",427.210,4.894901e+05,1361.340,-0.30
7,2021-04,"847,186",454.270,4.976600e+05,1405.530,-0.30
8,2021-03,"822,551",644.550,1.106713e+06,1441.130,0.60
9,2021-02,"813,680",661.990,6.186174e+05,1433.820,0.60


In [623]:
#Dataframe without Margin feature
Merged = pd.concat([BTCDrop, SNPDrop, CPIDrop], axis=1, join='inner')
Merged

,BTC,Liquidity,SNP close,CPI close
Date,,,,
2020-04-09,7315.25,16310014.53,2789.82,0.1
2020-04-08,7369.60,18138080.27,2749.98,0.1
2020-04-07,7201.81,17047120.32,2659.41,0.1
2020-04-06,7336.96,26533750.17,2663.68,0.1
2020-04-03,6740.88,19692803.68,2488.65,0.1
...,...,...,...,...
2015-10-14,253.00,30115.77,1994.24,-0.1
2015-10-13,250.00,12875.40,2003.69,-0.1
2015-10-12,247.60,9493.89,2017.46,-0.1
